In [1]:
import pandas as pd
from tqdm import tqdm
import json


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json

# ============================
# 1. Model Transformers
# ============================
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct-AWQ"

print("Đang load model Transformers...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={"": "cuda"},
    trust_remote_code=True
)

# ============================
# 2. Dữ liệu
# ============================
INPUT_FILE = "test_flow_reviews_1.xlsx"
COL_REVIEW = "reviewContent"
OUTPUT_FILE = "full_reviews_1.xlsx"

CANONICAL = [
    "Chất lượng sản phẩm",
    "Giá cả",
    "Vận chuyển & Giao hàng",
    "Đóng gói & Bao bì",
    "Dịch vụ & CSKH",
    "Mô tả & Hình ảnh",
    "Lỗi, bảo hành, hàng giả",
    "Trải nghiệm sử dụng",
    "Uy tín & thái độ shop",
    "Khuyến mãi & voucher"
]

df = pd.read_excel(INPUT_FILE)
df = df[3:8]
for asp in CANONICAL:
    df[asp] = 2   # mặc định = 2 = không đề cập

# ============================
# 3. Prompt
# ============================
SYSTEM_PROMPT = """
Bạn là AI phân tích cảm xúc cho đánh giá sản phẩm thương mại điện tử tiếng Việt.

Nhiệm vụ:
1. Trích xuất các "target" (khía cạnh) mà người dùng đang khen/chê trong câu review.
2. Chuẩn hóa mỗi target về một trong CÁC NHÓM CỐ ĐỊNH dưới đây (canonical aspect):

- "Chất lượng sản phẩm"
- "Giá cả"
- "Vận chuyển & Giao hàng"
- "Đóng gói & Bao bì"
- "Dịch vụ & CSKH"
- "Mô tả & Hình ảnh"
- "Lỗi, bảo hành, hàng giả"
- "Trải nghiệm sử dụng"
- "Uy tín & thái độ shop"
- "Khuyến mãi & voucher"

3. Gán score cho từng target: "1": "tích cực", "-1": "tiêu cực", "0": "trung lập" hoặc "2": "không đề cập".

Yêu cầu:
- Chỉ trả về JSON hợp lệ.
- Không giải thích, không thêm text ngoài JSON.
- Trả về JSON gồm đúng 10 keys.

Format JSON OUTPUT:

{
  "targets": [
    {
      "raw_target": "<target chi tiết, ví dụ: 'pin', 'camera', 'độ thoải mái'>",
      "canonical_aspect": "<một trong 10 nhóm canonical>",
      "sentiment": "<tích cực | tiêu cực | trung lập>"
    }
  ]
}
""".strip()


def build_prompt(review: str):
    return (
        f"<|im_start|>system\n{SYSTEM_PROMPT}\n<|im_end|>\n"
        f"<|im_start|>user\nReview: \"{review}\"\nHãy trả về JSON.\n<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )


# ============================
# 4. Chạy labeling từng dòng + tqdm
# ============================

print("🔥 Bắt đầu labeling bằng Transformers...\n")

sentiment_map = {
    "tích cực": 1,
    "tiêu cực": -1,
    "trung lập": 0,
    "không đề cập": 2
}

for idx in tqdm(df.index, desc="Labeling", unit="row"):

    review = str(df.at[idx, COL_REVIEW])
    prompt = build_prompt(review)

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.0,
        do_sample=False
    )

    # Chỉ decode phần generated (sau input)
    input_length = inputs.input_ids.size(1)
    generated = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)

    # Parse JSON từ generated
    try:
        start = generated.index("{")
        end = generated.rindex("}") + 1
        data = json.loads(generated[start:end])
    except:
        data = {}

    # reset tất cả canonical = 2 ở mỗi dòng
    # for asp in CANONICAL:
    #     df.at[idx, asp] = 2

    # Nếu JSON hợp lệ
    if isinstance(data, dict) and "targets" in data:
        for item in data["targets"]:
            canonical = item.get("canonical_aspect")
            senti = item.get("sentiment")

            if canonical in CANONICAL:
                df.at[idx, canonical] = sentiment_map.get(senti, 2)


# ============================
# 5. Lưu file
# ============================

df.to_excel(OUTPUT_FILE, index=False)
print("File đã lưu:", OUTPUT_FILE)

c:\Users\Luc\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Đang load model Transformers...


c:\Users\Luc\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luc\.cache\huggingface\hub\models--Qwen--Qwen2.5-1.5B-Instruct-AWQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
